# Imports

In [1]:
# Please update the default location as you deem fit
cvml_path = '/home/alex.li/git/JupiterCVML/europa/base/src/europa'

In [2]:
import json
import os
import multiprocessing as mp

import numpy as np
import pandas as pd

from cv.core.image_quality_server_side import ImageQuality
from dl.utils.config import DEFAULT_TONEMAP_PARAMS

## Datasets

In [3]:
dataset_idx = 0
directory = ['/data2/jupiter/datasets/20231017_halo_rgb_labeled_excluded_bad_iq', '/data/jupiter/datasets/iq_2023_v5_anno'][dataset_idx]
csv_name = ['653a7a0a3c2d8ab221f6d915_master_annotations.csv','64dfcc1de5a41169c7deb205_master_annotations.csv'][dataset_idx]

dset_name = directory.split('/')[-1]
save_path='/mnt/sandbox1/alex.li/iq_results'

side_left_tire_mask = f'{cvml_path}/cv/core/tire_masks/side_left_iq_mask.png'
side_right_tire_mask = f'{cvml_path}/cv/core/tire_masks/side_right_iq_mask.png'

# Prediction from JupiterCVML repo

In [4]:
iq = ImageQuality(num_workers=mp.cpu_count() // 2,
                  use_progress=True,
                  side_left_tire_mask_path = side_left_tire_mask,
                  side_right_tire_mask_path = side_right_tire_mask,
                  normalization_params=DEFAULT_TONEMAP_PARAMS,
                  dataset=dset_name,
                  save_path=save_path)
print(directory)
# stereo_df = pd.read_csv(os.path.join(directory, 'master_annotation_with_histogram.csv'), low_memory=False)
stereo_df = pd.read_csv(os.path.join(directory, csv_name), low_memory=False)

/data2/jupiter/datasets/20231017_halo_rgb_labeled_excluded_bad_iq


In [5]:
stereo_df = stereo_df.drop_duplicates(['id'])

In [6]:
out = pd.value_counts(stereo_df['camera_location'])
for i, v in zip(out.index, out.values):
    print(f"{i},{v}")

I02,392
I01,338
T14,35
T10,31
T05,18
T09,13
T13,9
T02,6
T06,6


In [7]:
labeled = iq.from_df(stereo_df, directory, use_progress=True)

In [9]:
labeled['iq'] = labeled.image_quality.apply(lambda x: x.algorithm_output)
labeled['iq_features'] = labeled.image_quality.apply(lambda x: x.algorithm_features)
labeled['iq_features_total'] = labeled.iq_features.apply(lambda x: x['image_features']['total'])
labeled['iq_features_low'] = labeled.iq_features.apply(lambda x: x['image_features']['low'])
labeled['iq_features_mid'] = labeled.iq_features.apply(lambda x: x['image_features']['mid'])
labeled['iq_features_high'] = labeled.iq_features.apply(lambda x: x['image_features']['high'])
labeled['iq_features_depth_ratio'] = labeled.iq_features.apply(lambda x: x['image_features']['depth_ratio'])
labeled['iq_features_smudge'] = labeled.iq_features.apply(lambda x: x['image_features']['smudge'])
labeled['iq_features_smudge_reason'] = labeled.iq_features.apply(lambda x: x['image_features']['smudge_reason'])
labeled['iq_process_time'] = labeled.image_quality.apply(lambda x: x.algorithm_process_time)

if 'iq_ground_truth' in labeled:
    labeled['binary_iq'] = labeled.iq.apply(lambda x: 'iq' if x != 'good' else 'non_iq')
    labeled['binary_iq_ground_truth'] = labeled.iq_ground_truth.apply(lambda x: 'iq' if x != 'good' else 'non_iq')

output_path = f'{save_path}/{dset_name}'
os.makedirs(output_path, exist_ok=True)
labeled.to_csv(f'{output_path}/iq.csv')

labeled.iq.value_counts()

smudge       403
bad_depth    354
good          87
dark           4
Name: iq, dtype: int64

In [14]:
print(f"{output_path}/iq.csv")
df = pd.read_csv(f"{output_path}/iq.csv")

/mnt/sandbox1/alex.li/iq_results/20231017_halo_rgb_labeled_excluded_bad_iq/iq.csv


In [27]:
print(','.join(sorted(list(set(df[(df.iq == 'good')]['id'])))))

64d2770bad589cfd4ee9041f,64d27ea4c5e3eebecd51bb7d,64d28010e4b5f60207a5611b,64d28029bcb05e3b0204ef6e,64d28036d22763890d9497dd,64d2804291294bf9055ca710,64d281a6bdfa4c027176ac15,64d281bdcdddad3864b81300,64d29b3edd91722b31bd270d,64d29f236ee204e85918d31b,64d2be6a46228d2623d7cc5b,64d2d56c84dac3a508bf58fa,64d2e2e79c4221cabc920839,64d300b4b45885cfe61c3158,64d3017643e80ba938649185,64d303f30e8be16822317063,64d3085b77a3f6f6309aefad,64d30d9b46228d2623d8750d,64d315c1073191b1e22464bd,64d3298a84dac3a508c00ec0,64d3347dcdddad3864b9c940,64d33496d22763890d965d17,64d334c0b45885cfe61c9ef5,64d335a054324d20f7e693e6,64d337dc77a3f6f6309b4820,64d33aff0416decb1dccb262,64d33b8e4210b6d080dd84e5,64d33bb84210b6d080dd8560,64d3452c0416decb1dccca71,64d356e84079e515f9344b38,64d357aa404d0689282f6c68,64d391758ebf422ddd91a88f,64d3a42a3379a4c77440ab65,64d3e90a31b7ca5821875b0e,64d3efc3404d068928302ff3,64d3f211c28c73a9401310e2,64d40923d22763890d982dc0,64d40a19aae42de8ded9d068,64d40e6e31b7ca582187b070,64d41ea8cdddad3864bbbe94,

In [24]:
pd.value_counts(df[df.iq == 'bad_depth']['camera_location'])

I02    186
I01    127
T14     13
T05     10
T09      8
T10      8
T13      2
Name: camera_location, dtype: int64

## Accuracy Metrics
#### Valid only if ground-truth image_quality_labels exist

In [39]:
def accuracy(df, iq_type='overall'):
    if iq_type!='overall':
        sub_df = df[df.iq_ground_truth==iq_type]
    else:
        sub_df = df
    valid = np.sum(sub_df.iq == sub_df.iq_ground_truth)
    total = len(sub_df)
    acc = valid/total if total != 0 else 0
    print(f'{iq_type}\tproperly-predicted:{valid:4d} \ttotal-images:{total:4d}\tacc:{100*acc:.3f}%')

if 'iq_ground_truth' in labeled:
    accuracy(labeled, 'smudge')  
    accuracy(labeled, 'dark')  
    accuracy(labeled, 'bright')  
    accuracy(labeled, 'good') 
    accuracy(labeled, 'overall')
    valid = np.sum(labeled.binary_iq == labeled.binary_iq_ground_truth)
    total = len(labeled)
    acc = valid/total
    print(f'IqNonIq\tproperly-predicted:{valid:4d} \ttotal-images:{total:4d}\tacc:{100*acc:.3f}%')

0      Colorado
1      Colorado
2      Colorado
3      Colorado
4      Colorado
         ...   
265     Georgia
266     Georgia
267     Georgia
268     Georgia
269     Georgia
Name: state, Length: 270, dtype: object